<a href="https://colab.research.google.com/github/jessepoljak/neural-network-challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [92]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [93]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [94]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()



,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [113]:
# Create a list of at least 10 column names to use as X data
columns = ['Age', 'Education', 'DistanceFromHome', 'JobSatisfaction', 'StockOptionLevel', 'WorkLifeBalance', 'YearsSinceLastPromotion', 'NumCompaniesWorked', 'JobInvolvement', 'JobLevel']



# Create X_df using your selected columns
X_df = attrition_df[columns]
X_df.head()


# Show the data types for X_df
X_df.dtypes



,0
Age,int64
Education,int64
DistanceFromHome,int64
JobSatisfaction,int64
StockOptionLevel,int64
WorkLifeBalance,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64
JobInvolvement,int64
JobLevel,int64


In [114]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=3)

# Scale the X data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [115]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

X_df.dtypes


#All X data is already numeric


,0
Age,int64
Education,int64
DistanceFromHome,int64
JobSatisfaction,int64
StockOptionLevel,int64
WorkLifeBalance,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64
JobInvolvement,int64
JobLevel,int64


In [116]:
# Create a StandardScaler
scaler = StandardScaler()


# Fit the StandardScaler to the training data
scaler.fit(X_train)


# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [117]:
# Create a OneHotEncoder for the Department column
# Fit the encoder to the training data
from sklearn.preprocessing import OneHotEncoder
dept_encoder = OneHotEncoder(sparse_output=False)
dept_encoded = dept_encoder.fit_transform(y_train[['Department']])
dept_encoded_df = pd.DataFrame(dept_encoded, columns=dept_encoder.get_feature_names_out(['Department']))
dept_encoded_df.head()

# Create two new variables by applying the encoder
# to the training and testing data
dept_encoded_train = dept_encoder.transform(y_train[['Department']])
dept_encoded_test = dept_encoder.transform(y_test[['Department']])





In [118]:
# Create a OneHotEncoder for the Attrition column
att_encoder = OneHotEncoder(sparse_output=False)
att_encoded = att_encoder.fit_transform(y_train[['Attrition']])
att_encoded_df = pd.DataFrame(att_encoded, columns=att_encoder.get_feature_names_out(['Attrition']))
att_encoded_df.head()



# Fit the encoder to the training data
# Create two new variables by applying the encoder
# to the training and testing data
att_encoded_train = att_encoder.transform(y_train[['Attrition']])
att_encoded_test = att_encoder.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [119]:
# Find the number of columns in the X training data
num_columns = X_train_scaled.shape[1]
num_columns


# Create the input layer
input_layer = layers.Input(shape=(num_columns,))


# Create at least two shared layers
shared_layer1 = layers.Dense(units=6, activation="relu")(input_layer)
shared_layer2 = layers.Dense(units=6, activation="relu")(shared_layer1)


In [120]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden = layers.Dense(units=6, activation="relu", name="dept_hidden")(shared_layer2)

# Create the output layer
dept_output = layers.Dense(units=3, activation="softmax", name="dept_output")(dept_hidden)


In [130]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(units=6, activation="relu", name="attrition_hidden")(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(units=2, activation="sigmoid", name="attrition_output")(attrition_hidden)


In [131]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attrition_output])


# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics={'dept_output': 'accuracy', 'attrition_output': 'accuracy'}
)

# Summarize the model
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 6)              │             66 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 6)              │             42 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dept_hidden (Dense)       │ (None, 6)              │             42 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 6)              │             42 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dept_output (Dense)       │ (None, 3)              │             21 │ dept_hidden[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             14 │ attrition_hidden[0][0] │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 227 (908.00 B)

 Trainable params: 227 (908.00 B)

 Non-trainable params: 0 (0.00 B)

In [132]:
# Train the model
model.fit(X_train_scaled, [dept_encoded_train, att_encoded_train], epochs=100, batch_size=32)



Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - attrition_output_accuracy: 0.5104 - attrition_output_loss: 0.7295 - dept_output_accuracy: 0.6101 - dept_output_loss: 0.7773 - loss: 1.5070
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.7728 - attrition_output_loss: 0.6173 - dept_output_accuracy: 0.6241 - dept_output_loss: 0.7764 - loss: 1.3937
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8312 - attrition_output_loss: 0.5399 - dept_output_accuracy: 0.6431 - dept_output_loss: 0.7552 - loss: 1.2954
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8393 - attrition_output_loss: 0.4729 - dept_output_accuracy: 0.6329 - dept_output_loss: 0.7420 - loss: 1.2147
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8433 - attrition_output_loss: 0.4392 - dept_output_accuracy: 0.6399 - dept_output_loss: 0.7335 - loss: 1.1729
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0

In [133]:
# Evaluate the model with the testing data
total_loss, dept_loss, dept_accuracy, attrition_loss, attrition_accuracy = model.evaluate(X_test_scaled, [dept_encoded_test, att_encoded_test])




12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8543 - attrition_output_loss: 0.3965 - dept_output_accuracy: 0.6573 - dept_output_loss: 0.7729 - loss: 1.1689  


In [134]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {dept_accuracy}")
print(f"Attrition Accuracy: {attrition_accuracy}")


Department Accuracy: 0.4141150414943695
Attrition Accuracy: 0.6521739363670349


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is likely not the best metric for the dataset on it's own. For instance, there is definitely imbalance with attrition, with many more people not experiencing it. This can make it much more difficult to train the model to predict attrition vs. non-attrition since it has way more examples of one of them.

2. Softmax for department given it is a multi-class variable. For Attrition, I used sigmoid, since it is a binary variable.

3. Having more data, balancing the data, and testing more layers and activation methods may help improve the model to make it more accurate. In addition, I'd want to look at some other non-accuracy metrics to more completely measure results.